# Python Concepts Selected

This notebook covers some miscalleneous Python concepts and questions that I thought they can be interesting.

# Table of Contents

- [\_\_init\_\_.py](#__init__py)
- [Python Packages](#python-packages)
- [Serialization](#serialization)
- [\_\_str\_\_ vs \_\_repr\_\_](#__str__-vs-__repr__)
- [Hashable Objects](#hashable-objects)
- [Context Managers and the with Statement](#context-managers-and-the-with-statement)
- [Abstraction vs. Encapsulation](#abstraction-vs-encapsulation)
- [Multiple Inheritance and the Diamond Problem](#multiple-inheritance-and-the-diamond-problem)
- [Compiled Bytecode in Python](#compiled-bytecode-in-python)
- [Arrays in Python](#arrays-in-python)
- [Accessing Private Variables in Subclasses (Mangling)](#accessing-private-variables-in-subclasses-mangling)
- [Shallow Copy vs. Deep Copy](#shallow-copy-vs-deep-copy)
- [The id() Function](#the-id-function)
- [Container Methods](#container-methods)
- [Differences Between Regular and Lambda Functions](#differences-between-regular-and-lambda-functions)
- [Global Interpreter Lock (GIL)](#global-interpreter-lock-gil)
- [dundr Methods](#dundr-methods)
- [new Method](#new-method)
- [Monkey patching](#monkey-patching)
- [Duck Typing in Python?](#duck-typing-in-python)
- [What Python supports, call by reference or call by value?](#what-python-supports-call-by-reference-or-call-by-value)
- [MRO](#mro)
- [References in Python](#references-in-python)
- [Enumerations](#enumerations)



## \_\_init\_\_.py

The `__init__.py` file is an essential component of Python packages. Its primary purpose is to initialize a Python package and can serve several important functions:

1. **Package Initialization**: Indicates that the directory should be treated as a package.
2. **Namespace Management**: Manages the namespace of the package.
3. **Package-level Variables and Functions**: Can define package-level variables and functions.
4. **Simplifying Imports**: Makes it easier for users to import modules or functions.

## Python Packages

A Python package is a way of organizing and structuring Python code in a hierarchical manner. A package is a directory that contains Python modules and a special `__init__.py` file.

### Key Features

- **Hierarchy**: Allows for hierarchical structuring of the namespace.
- **Namespacing**: Avoids naming conflicts between modules.
- **Reusability**: Makes it easy to share and distribute Python code.
- **Modularity**: Promotes modularity by segregating code into distinct modules.


### Structure

```plaintext
mypackage/
    __init__.py
    module1.py
    module2.py
    subpackage1/
        __init__.py
        submodule1.py
```


### Using Packages

```python
# Importing a module from a package
from mypackage import module1

# Importing specific functions or classes from a module
from mypackage.module1 import my_function
```

## Serialization

Serialization is the process of converting a Python object into a format that can be stored on disk or transmitted over a network. This format can be a byte stream, a binary file, or a text file (like JSON or XML), depending on the serialization method used. The main goal of serialization is to save the state of an object so that it can be reconstructed later, either in the same or a different environment.

1. **Binary Format**: When objects are serialized into binary format (using modules like `pickle` in Python), they are converted into a byte stream that can represent complex Python objects, including instances of custom classes, with their internal state and data structure preserved.

2. **Textual Format**: Serialization can also produce a text-based format like JSON or XML. These formats are more human-readable and can be easily shared across different programming environments. However, they might not support all Python-specific data types directly and often require conversion into types that are more universally recognized (e.g., lists, dictionaries, strings, numbers).

### Binary Serialization with `pickle`

The `pickle` module in Python is used for serializing and deserializing objects.

In [119]:
import pickle

# Example object
obj = {'key': 'value'}

# Serializing the object
with open('data/obj.pickle', 'wb') as file:
    pickle.dump(obj, file)

# Deserializing the object
with open('data/obj.pickle', 'rb') as file:
    loaded_obj = pickle.load(file)
    print(loaded_obj) 

{'key': 'value'}


### Textual Serialization with `json`

The `json` module is part of the Python Standard Library and provides functions to serialize and deserialize objects to and from JSON (JavaScript Object Notation) format.

**Serializing Objects**

To serialize an object, you can use the `json.dumps()` function, which takes an object as an argument and returns a JSON-encoded string:

In [120]:
import json

data = {'name': 'John', 'age': 30}

json_string = json.dumps(data)
print(json_string) 

{"name": "John", "age": 30}


**Deserializing Objects**

To deserialize a JSON-encoded string, you can use the `json.loads()` function, which takes a JSON-encoded string as an argument and returns a Python object:

In [121]:
json_string = '{"name": "Jane", "age": 25}'
data = json.loads(json_string)
print(data)

{'name': 'Jane', 'age': 25}


**Custom Serialization**

If you need to serialize custom objects, you can create a custom `JSONEncoder` class to handle the serialization:

In [122]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

class PersonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Person):
            return {'name': obj.name, 'age': obj.age}
        return super().default(obj)

person = Person('Alice', 30)
json_string = json.dumps(person, cls=PersonEncoder)
print(json_string)

{"name": "Alice", "age": 30}


## \_\_str\_\_ vs \_\_repr\_\_

The `__str__` and `__repr__` methods are used for object representation, serving different purposes and contexts.

### `__repr__`

- **Purpose**: To be unambiguous and, ideally, return a string that could recreate the object.
- **Context**: Used mainly for debugging and development.

**Example**

In [123]:
class MyClass:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f'MyClass({self.value!r})'

obj = MyClass(5)
print(repr(obj))

MyClass(5)


### `__str__`

- **Purpose**: To be readable and return a nicely printable string representation.
- **Context**: Invoked by the `str()` function and `print` statement.

**Example**

In [124]:
class MyClass:
    def __init__(self, value):
        self.value = value

    def __str__(self):
        return f'Value is {self.value}'

obj = MyClass(5)
print(str(obj)) 

Value is 5


### Key Differences

- `__repr__` is for developers, providing a detailed representation.
- `__str__` is for end users, offering a friendly representation.
- If `__str__` is not defined, Python will use `__repr__`.

## Hashable Objects

In Python, if an object is hashable, it means that it has a hash value that does not change during its entire lifetime (it needs a `__hash__()` method), and it can be compared to other objects (it needs an `__eq__()` or `__cmp__()` method). Hashability makes an object usable as a dictionary key and a set member because these data structures use the hash value internally.

Most of Python's immutable built-in objects are hashable; mutable containers (such as lists or dictionaries) are not hashable, while immutable containers (like tuples and frozensets) are hashable if all their elements are hashable.

Here are some points about hashability in Python:

1. **Immutable Types**: Simple immutable types (like `int`, `float`, `str`, and `frozenset`) are hashable. For example, once you create an integer in Python, its hash value will never change, making it possible to reliably use it as a key in a dictionary or as a member of a set.

2. **Custom Objects**: By default, instances of user-defined classes are hashable because their hash value is their `id()` and they all compare not equal (except with themselves).

3. **Mutability**: If an object is mutable, like a list, it cannot be hashable. This is because if the object could be changed after it was added to a set or used as a key in a dictionary, it would be in the wrong hash bucket and could not be reliably found.

4. **Consistency**: For an object to be hashable, it must have a hash value that remains the same during its lifetime. This hash value is obtained using the `__hash__()` method. The object also needs to compare equal to other objects that have the same hash value using the `__eq__()` method.

5. **Custom `__hash__` Implementation**: If you define a custom `__eq__` method in a class, you must also define a custom `__hash__` method. It should return an integer and ideally take into account the same object attributes that are used for equality (`__eq__`). If `__eq__` is defined but `__hash__` is not, the instances of the class will not be hashable.

To summarize, hashability in Python is a property of objects that are intended to be constant and used as the lookup key for sets and dictionaries.

### Custom Hashable Class Example

In [125]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.x, self.y))

point1 = Point(2, 3)
point2 = Point(2, 3)

points_set = {point1}
print(point2 in points_set)

points_dict = {point1: 'A'}
print(points_dict[point2])

True
A


In this class:
- `__init__` initializes the point with x and y coordinates.
- `__eq__` ensures that two points are considered equal if their coordinates are the same.
- `__hash__` returns a hash that is a tuple of the point's coordinates.

The `Point` class instances can be used as dictionary keys and stored in sets because they are hashable.

## Context Managers and the `with` Statement

Context managers allow you to allocate and release resources precisely when you want to. The most common example is file handling.

### Basic Example of `with` for File Handling:

In [126]:
with open('data/large_log_file.txt', 'r') as file:
    contents = file.read()

### Custom Context Manager using a Class:

In [127]:
class Open_File:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.file.close()

with Open_File('data/large_log_file.txt', 'r') as file:
    contents = file.read()

### Custom Context Manager using `contextlib`:

In [128]:
from contextlib import contextmanager

@contextmanager
def open_file(name, mode):
    f = open(name, mode)
    try:
        yield f
    finally:
        f.close()

with open_file('data/large_log_file.txt', 'r') as file:
    contents = file.read()

## Abstraction vs. Encapsulation

### Abstraction

Abstraction is the concept of hiding the complex reality while exposing only the necessary parts. It focuses on the interface rather than the implementation.

**Example:**

In [129]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    @abstractmethod
    def start(self):
        pass

    @abstractmethod
    def stop(self):
        pass

class Car(Vehicle):
    def start(self):
        print("Car engine starts.")

    def stop(self):
        print("Car brakes activate.")

### Encapsulation

Encapsulation is the concept of wrapping data and methods that operate on the data within a single unit, usually a class, and restricting access to some of the object's components.

**Example:**

In [130]:
class BankAccount:
    def __init__(self, initial_balance=0):
        self.__balance = initial_balance  # Private variable

    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount

    def withdraw(self, amount):
        if 0 < amount <= self.__balance:
            self.__balance -= amount

    def get_balance(self):
        return self.__balance

account = BankAccount(100)
account.deposit(50)
account.withdraw(20)
print(account.get_balance())

130


## Multiple Inheritance and the Diamond Problem

Python supports multiple inheritance, which can lead to the diamond problem. The diamond problem occurs when a class inherits from two classes that both inherit from a common ancestor.

### Example of Multiple Inheritance:

In [131]:
class A:
    def do_something(self):
        print("Method defined in A")

class B(A):
    def do_something(self):
        print("Method defined in B")

class C(A):
    def do_something(self):
        print("Method defined in C")

class D(B, C):
    pass

d = D()
d.do_something()

Method defined in B


### Method Resolution Order (MRO)

Python resolves the diamond problem using the C3 linearization algorithm. The MRO can be inspected using the `__mro__` attribute or the `mro()` method.

In [132]:
print(D.__mro__)
# or
print(D.mro())

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]



When `d.do_something()` is called, the output will be:

```
Method defined in B
```

Because `B` comes before `C` in the MRO of class `D`.

## Compiled Bytecode in Python


Python is often referred to as an interpreted language, but it actually goes through a compilation step before execution. This involves compiling Python source code (`.py` files) into bytecode (`.pyc` files).

### What is Bytecode?

- **Bytecode** is a low-level, platform-independent representation of your source code.
- **.pyc Files**: These files contain the compiled bytecode and are stored in the `__pycache__` directory. This step speeds up the program startup time because the bytecode doesn't need to be recompiled each time the script is run.

### How Bytecode Works:

1. **Compilation**: When you run a Python program, the Python interpreter compiles the source code into bytecode.
2. **Execution**: The bytecode is then executed by the Python virtual machine (PVM).

```python
# Example: Running a Python script
# Compiles the script into bytecode and saves it in __pycache__
python example.py
```

## Arrays in Python

While Python lists are flexible and can hold different data types, sometimes a more efficient, typed array is needed. This is where the `array` module and libraries like NumPy come in.

### Python's Built-in `array` Module

The `array` module provides a way to create arrays with fixed types.

In [133]:
import array

# Create an array of integers
int_array = array.array('i', [1, 2, 3, 4])
print(int_array)

# Add an element
int_array.append(5)
print(int_array)

# Iterate over the array
for item in int_array:
    print(item, end=' ')

array('i', [1, 2, 3, 4])
array('i', [1, 2, 3, 4, 5])
1 2 3 4 5 

### NumPy Arrays

NumPy provides a powerful array object, `ndarray`, which supports multi-dimensional arrays and a variety of mathematical operations.

In [134]:
import numpy as np

# Create a NumPy array
np_array = np.array([1, 2, 3, 4, 5])
print(np_array)

# Perform operations
np_array = np_array * 2
print(np_array)

# Multi-dimensional array
np_matrix = np.array([[1, 2], [3, 4]])
print(np_matrix)

[1 2 3 4 5]
[ 2  4  6  8 10]
[[1 2]
 [3 4]]


## Accessing Private Variables in Subclasses (Mangling)

In Python, private variables are indicated by prefixing the name with double underscores (`__`). These are subject to name mangling, which makes them less accessible from outside the class.

In [135]:
class Parent:
    def __init__(self):
        self._protectedVar = "I'm protected"
        self.__privateVar = "I'm private"

    def _protectedMethod(self):
        return "Protected method"

    def __privateMethod(self):
        return "Private method"

class Child(Parent):
    def accessProtected(self):
        return self._protectedVar

    def accessPrivate(self):
        # Accessing private variable using name mangling
        return self._Parent__privateVar

p = Parent()
print(p._protectedVar)  # Directly accessible

c = Child()
print(c.accessProtected()) 
print(c.accessPrivate())

I'm protected
I'm protected
I'm private


## Shallow Copy vs. Deep Copy

### Shallow Copy

A shallow copy creates a new object but does not create copies of nested objects. Instead, it copies the references to the nested objects.

In [136]:
import copy

original_list = [[1, 2, 3], [4, 5, 6]]
shallow_copied_list = copy.copy(original_list)

original_list[0][0] = 'X'
print(shallow_copied_list) 

[['X', 2, 3], [4, 5, 6]]


### Deep Copy

A deep copy creates a new object and recursively copies all objects found in the original object, creating independent copies of all nested objects.

In [137]:
import copy

original_list = [[1, 2, 3], [4, 5, 6]]
deep_copied_list = copy.deepcopy(original_list)

original_list[0][0] = 'X'
print(deep_copied_list)  # Output: [[1, 2, 3], [4, 5, 6]]

[[1, 2, 3], [4, 5, 6]]


## The `id()` Function

The `id()` function returns the "identity" of an object, which is a unique integer for the object during its lifetime. It typically represents the object's memory address in CPython.

In [138]:
a = 'example'
print(id(a))  # Outputs a unique integer

b = 'example!'
print(id(b))  # Outputs a different unique integer

4721297136
4757822832


### Difference Between `id()` and `hash()`

- **id()**: Provides a unique identifier for the object during its lifetime.
- **hash()**: Returns a hash value used for quick comparisons in hash-based collections like dictionaries and sets.

In [139]:
a = 'example'
print(id(a))   # Unique identity
print(hash(a)) # Hash value

b = 'example!'
print(id(b))   # Different identity
print(hash(b)) # Different hash value, but not guaranteed

4721297136
6394412552866170590
4757905840
1890314095180015450


## Container Methods


Container methods allow custom objects to support common container operations such as indexing, length retrieval, item setting, and item deletion. Let's implement these methods in a class called `MyContainer`.


### Implementing Container Methods in `MyContainer`


1. **`__len__(self)`**: Returns the length of the container. Called by the `len()` function.
2. **`__getitem__(self, key)`**: Defines behavior for accessing an item `x[key]`.
3. **`__setitem__(self, key, value)`**: Defines behavior for setting an item `x[key] = value`.
4. **`__delitem__(self, key)`**: Defines behavior for deleting an item `del x[key]`.

Here's a complete example:

In [140]:
class MyContainer:
    def __init__(self):
        self.items = []

    def __len__(self):
        return len(self.items)

    def __getitem__(self, key):
        return self.items[key]

    def __setitem__(self, key, value):
        if key >= len(self.items):
            # Extend the list with None until we reach the desired index
            self.items.extend([None] * (key + 1 - len(self.items)))
        self.items[key] = value

    def __delitem__(self, key):
        # Set the item at key to None to indicate deletion
        # This is a simple approach; there are other ways to handle deletion
        self.items[key] = None

# Example usage
container = MyContainer()

# Adding items
container[0] = 'Python'
container[1] = 'is'
container[2] = 'awesome'

print(len(container))  
print(container[1])  

# Updating an item
container[1] = 'is really'
print(container[1])  

# Deleting an item
del container[1]
print(container[1])  

# Adding a new item at a higher index
container[4] = 'fun'
print(len(container))  

3
is
is really
None
5


### Explanation

1. **Initialization (`__init__`)**:
   - `self.items` is initialized as an empty list to hold the container's items.

2. **Length (`__len__`)**:
   - `__len__` returns the length of `self.items`, allowing `len(container)` to work.

3. **Get Item (`__getitem__`)**:
   - `__getitem__` returns the item at the given index `key`, enabling syntax like `container[key]`.

4. **Set Item (`__setitem__`)**:
   - `__setitem__` sets the value at the specified index `key`. If the index is out of bounds, it extends the list with `None` values up to the required length.

5. **Delete Item (`__delitem__`)**:
   - `__delitem__` sets the item at the specified index `key` to `None`, simulating deletion.


## Differences Between Regular and Lambda Functions 

In Python, anonymous functions are functions that are defined without a name. While regular functions are defined using the `def` keyword, anonymous functions are defined using the `lambda` keyword.

Here are the key differences between a normal (named) function and an anonymous (lambda) function:

1. **Definition Syntax:**
   - Normal functions are defined using the `def` keyword, have a name, and can consist of multiple statements.
   - Lambda functions are defined using the `lambda` keyword, do not have a name, and consist of a single expression.

2. **Number of Expressions:**
   - Normal functions can contain any number of expressions and statements.
   - Lambda functions are limited to a single expression.

3. **Return Statement:**
   - Normal functions can use the `return` statement.
   - Lambda functions implicitly return the result of their single expression.

4. **Use Case:**
   - Normal functions are used when you need to perform a sequence of operations and possibly reuse this functionality.
   - Lambda functions are used for small, one-off operations that you can define inline, often where a function expects a callback parameter.

Here are examples that demonstrate the differences:

In [141]:
def add(a, b):
    result = a + b
    return result

print(add(2, 3))

5


In [142]:
add = lambda a, b: a + b

print(add(2, 3))

5


In the examples above, both functions achieve the same result, but the lambda function does it in a single line. Lambda functions are particularly useful when you need a small function for a short duration and you are interested in quickly passing a function as an argument to higher-order functions like `map()`, `filter()`, or `sorted()`.

Keep in mind that lambda functions are syntactically restricted to a single expression. So, you can't have multi-line statements, loops, or complex logic inside a lambda function. For these more complex cases, you would define a normal function using `def`.

1. **Using `map()` with a lambda function:**

The `map()` function applies a given function to each item of an iterable (like a list) and returns a list of the results.


In [143]:
# Example using map() to square each number in the list
numbers = [1, 2, 3, 4, 5]
squared_numbers = map(lambda x: x**2, numbers)

print(list(squared_numbers))


[1, 4, 9, 16, 25]


2. **Using `filter()` with a lambda function:**

The `filter()` function creates a list of elements for which a function returns true.


In [144]:
# Example using filter() to get only even numbers from the list
numbers = [1, 2, 3, 4, 5]
even_numbers = filter(lambda x: x % 2 == 0, numbers)

print(list(even_numbers))

[2, 4]


3. **Using `sorted()` with a lambda function:**

The `sorted()` function sorts the items of a given iterable in a specific order (default is ascending) and returns a sorted list.

In [145]:
# Example using sorted() to sort a list of tuples based on the second item
pairs = [(1, 'one'), (3, 'three'), (2, 'two'), (4, 'four')]
sorted_pairs = sorted(pairs, key=lambda x: x[1])

print(sorted_pairs)


[(4, 'four'), (1, 'one'), (3, 'three'), (2, 'two')]


Lambda functions can use conditional expressions to incorporate `if-else` statements within a single line of code. The syntax for using `if-else` in a lambda function is:

```python
lambda arguments: expression1 if condition else expression2
```

Here's an example of a lambda function that uses an `if-else` statement to determine whether a number is odd or even:

In [146]:
is_even = lambda x: "Even" if x % 2 == 0 else "Odd"

print(is_even(10)) 
print(is_even(3)) 

Even
Odd


You can also chain multiple `if-else` statements together, although it can make the lambda function harder to read:

In [147]:
classify_number = lambda x: "Positive" if x > 0 else ("Negative" if x < 0 else "Zero")

print(classify_number(10))
print(classify_number(-5))
print(classify_number(0))

Positive
Negative
Zero


While lambda functions are useful for simple conditional logic, for more complex scenarios with multiple conditions, it is often better to use a standard function definition with `def` for the sake of clarity.

## Global Interpreter Lock (GIL)

The Global Interpreter Lock (GIL) is a mutex that protects access to Python objects, preventing multiple threads from executing Python bytecode at once. This lock is necessary because CPython's memory management is not thread-safe. The GIL is part of Python's reference implementation, CPython, which is the most widely used implementation of Python.

### Why the GIL Exists


The GIL simplifies the CPython implementation by making the object model (including critical built-in types such as lists and dictionaries) inherently safe against concurrent access. This means that developers don’t need to worry about locking Python objects (with some exceptions) to ensure thread safety in their Python code. However, the presence of the GIL has significant implications for multi-threaded Python programs:

1. **CPU-Bound Tasks**: For CPU-bound tasks that require heavy computation and would benefit from parallel execution across multiple CPU cores, the GIL is a bottleneck. Even though these tasks can be executed in a multi-threaded manner, the GIL ensures that only one thread executes Python bytecode at a time. As a result, such tasks do not see a performance improvement from threading and might even perform worse due to the overhead of context switching between threads.

2. **I/O-Bound Tasks**: For I/O-bound tasks that spend most of their time waiting for external events (such as network responses or disk I/O), the GIL is less of a problem. While one thread waits for an I/O operation to complete, other threads can continue executing Python bytecode. Thus, multi-threading can improve the performance of I/O-bound applications by allowing them to handle other tasks while waiting for I/O operations.

### Workarounds and Alternatives

Despite the limitations imposed by the GIL, there are several approaches to achieve concurrency in Python:

1. **Multiprocessing**: By using the `multiprocessing` module, Python can bypass the GIL by creating separate processes instead of threads. Each process has its own Python interpreter and memory space, thus allowing parallel execution on multiple CPU cores. This is particularly effective for CPU-bound tasks.

2. **Alternative Python Implementations**: Some implementations of Python, such as Jython and IronPython, do not have a GIL. However, these implementations have their own limitations and may not be compatible with all Python code and libraries, especially those that rely on C extensions.

3. **Concurrent I/O**: For I/O-bound tasks, the `asyncio` module provides a framework for writing single-threaded concurrent code using coroutines. This model can handle high levels of I/O-bound tasks efficiently without the need for multi-threading or multiprocessing.

In summary, the GIL ensures thread safety within the CPython interpreter at the cost of limiting parallel execution of CPU-bound tasks in multi-threaded programs. Understanding the nature of the tasks your program performs is key to choosing the right concurrency model in Python.

## dundr Methods

In Python, "dunder" methods, short for "double underscore" methods, are special methods that start and end with double underscores (`__`). These methods are also known as "magic methods." They enable operator overloading, provide special object behaviors, and allow classes to integrate seamlessly with Python's built-in features. Below are some of the most commonly used dunder methods in a typical Python class:

### Object Initialization and Destruction

- `__init__(self, [...])`: The constructor method called when a new instance of the class is created. It can take arguments to initialize the instance.
- `__del__(self)`: The destructor method called when an instance is about to be destroyed. Though not frequently used, it can be helpful for cleaning up resources.

### Representation

- `__repr__(self)`: Called by the `repr()` built-in function and in many other contexts where a developer-friendly string representation of the object is needed.
- `__str__(self)`: Called by the `str()` function and by the `print()` function to compute the "informal" or nicely printable string representation of an object.

### Comparison Operators

- `__eq__(self, other)`: Defines behavior for the equality operator, `==`.
- `__ne__(self, other)`: Defines behavior for the inequality operator, `!=`.
- `__lt__(self, other)`: Defines behavior for the less than operator, `<`.
- `__le__(self, other)`: Defines behavior for the less than or equal to operator, `<=`.
- `__gt__(self, other)`: Defines behavior for the greater than operator, `>`.
- `__ge__(self, other)`: Defines behavior for the greater than or equal to operator, `>=`.

### Arithmetic Operators

- `__add__(self, other)`: Implements addition `+`.
- `__sub__(self, other)`: Implements subtraction `-`.
- `__mul__(self, other)`: Implements multiplication `*`.
- `__truediv__(self, other)`: Implements division `/`.
- `__floordiv__(self, other)`: Implements floor division `//`.
- `__mod__(self, other)`: Implements modulo `%`.
- `__pow__(self, other[, modulo])`: Implements power `**`.
- `__neg__(self)`: Implements unary negation `-`.
- `__pos__(self)`: Implements unary positive `+`.

### Container Methods

- `__len__(self)`: Returns the length of the container. Called by the `len()` function.
- `__getitem__(self, key)`: Defines behavior for accessing an item `x[key]`.
- `__setitem__(self, key, value)`: Defines behavior for setting an item `x[key] = value`.
- `__delitem__(self, key)`: Defines behavior for deleting an item `del x[key]`.
- `__iter__(self)`: Should return an iterator for the container's items, making the object iterable.

### Context Managers

- `__enter__(self)`: Enters the runtime context related to the object. The value returned by this method is bound to the identifier in the `as` clause of the `with` statement.
- `__exit__(self, exc_type, exc_value, traceback)`: Exits the runtime context and optionally handles exceptions.

## __new__ Method

The `__new__` method in Python is a special static method that is responsible for creating a new instance of a class. It precedes the `__init__` method in the object creation process and is unique because it's the actual constructor of the class, whereas `__init__` is the initializer that configures the newly created object instance.

### Purpose and Behavior
- **Object Creation**: `__new__` is responsible for returning a new instance of the class. It is called first when an object is created, before `__init__`.
- **Static Method**: Unlike most other dunder methods, `__new__` is a static method, which means it doesn't require an instance of the class. The first argument to `__new__` is the class itself, typically referred to as `cls`.

### Signature
```python
def __new__(cls, *args, **kwargs):
```
- `cls`: The class of which an instance was requested.
- `*args`, `**kwargs`: Additional arguments and keyword arguments that are passed through from the class instantiation call.

### Usage
`__new__` is rarely overridden, but there are specific cases where it's useful:
- **Immutable Object Customization**: When creating instances of immutable types like strings or tuples, where you cannot modify the instance after it's created, `__new__` can be used to customize the instance during creation.
- **Singleton Pattern Implementation**: To ensure a class only has one instance, `__new__` can control the instantiation process to return the same instance every time.
- **Returning Instances of Other Classes**: `__new__` can decide not to create a new instance of its own class, but rather to return an instance of a different class.


In [148]:
class Singleton:
    _instance = None  # Keep instance reference
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)  # Create a new instance if one doesn't exist
        return cls._instance  # Return the singleton instance
    
    def __init__(self, name):
        self.name = name

# Usage
singleton1 = Singleton('Instance 1')
singleton2 = Singleton('Instance 2')

print(singleton1 is singleton2)
print(singleton1.name)
print(singleton2.name)

True
Instance 2
Instance 2


In this example, `__new__` ensures that only one instance of `Singleton` is created, regardless of how many times the class is instantiated. The `__init__` method will still be called each time the class is instantiated, which is why the `name` attribute reflects the most recent instantiation argument.


## Monkey patching

Monkey patching in Python refers to the dynamic modification of a class or module at runtime. This technique allows the alteration of behavior of code without modifying the source code directly. It's often used in testing to change methods and attributes for mock testing or to fix bugs in a library without altering the external library's code.

Monkey patching is a powerful tool but should be used with caution, as it can lead to code that is hard to understand and maintain. It can also cause compatibility issues if the library being patched changes its implementation.

**Example**
Suppose you have a library module `mathlib` with a method `add` that you want to modify for your application's specific needs without changing the `mathlib` source code.

In [149]:
# Let's think it as the original `mathlib` module:
def add(a, b):
    return a + b

# Your application code:
# from mathlib import add

# Original behavior
print(f"Original add function's result is: {add(1, 2)}")

# Now, let's monkey-patch the `add` function.
def patched_add(a, b):
    # New behavior: add an extra 1 to the result
    return a + b + 1

# Patching
add = patched_add

# New behavior
print(f"Monkey patched add function's result is: {add(1, 2)}")


Original add function's result is: 3
Monkey patched add function's result is: 4


In this example, the `patched_add` function replaces the original `add` function in the `mathlib` module, altering its behavior for any code that uses `mathlib.add` after the patching. 

## Duck Typing in Python?

Duck typing is a concept related to dynamic typing in programming languages like Python, where an object's suitability for use is determined by the presence of certain methods and properties, rather than the actual type of the object itself. The name comes from the phrase "If it looks like a duck, swims like a duck, and quacks like a duck, then it probably is a duck," which emphasizes actions that an object can perform over its type.

In the context of Python, duck typing allows for more flexible and intuitive code. It means that you don't have to inherit from a specific class or implement a specific interface to fulfill an interface requirement, as long as your class has the necessary methods or attributes implemented.

**Example**

Consider a function that takes an object and calls its `walk` and `talk` methods. In Python, any object passed to this function just needs to have `walk` and `talk` methods. It doesn't matter what class the object is an instance of.

In [150]:
def activate_duck(duck):
    duck.walk()
    duck.talk()

class Duck:
    def walk(self):
        print("This duck is walking.")

    def talk(self):
        print("This duck says quack.")

class Person:
    def walk(self):
        print("This person is walking.")

    def talk(self):
        print("This person says hello.")

# Both objects can be used in the activate_duck function
duck = Duck()
person = Person()

activate_duck(duck)

activate_duck(person)

This duck is walking.
This duck says quack.
This person is walking.
This person says hello.


In the above example, the `activate_duck` function can accept any object as long as it has the `walk` and `talk` methods, demonstrating the principle of duck typing. This leads to a more flexible and extensible code, where functions can operate on a variety of objects without being tightly coupled to specific class hierarchies.

### Advantages of Duck Typing

- **Flexibility**: Allows for more generic and reusable code.
- **Simplicity**: Reduces the need for complex class hierarchies or interfaces.
- **Ease of Use**: Simplifies the design of functions and methods that can operate on a wide variety of objects.

### Considerations

While duck typing enhances flexibility and reduces boilerplate, it requires careful documentation and testing, as the implicit contracts it creates are not enforced by the language's syntax. Errors related to missing methods or incorrect implementations may only be discovered at runtime, making thorough testing and clear documentation essential.

## What Python supports, call by reference or call by value?

Python's argument passing model is neither strictly "call by value" nor "call by reference," but rather a strategy often referred to as "call by object reference" or "call by sharing." This means that when you pass an argument to a function in Python, what gets passed is a reference to the object, not the actual object itself. However, the way that changes to the object inside the function affect the original object outside the function depends on the object's type (mutable or immutable).

### Immutable Objects

Immutable objects (e.g., integers, floats, strings, tuples) cannot be changed after they are created. So, when you pass an immutable object to a function, and that function attempts to modify the "value" of the object, what actually happens is that a new object is created and assigned to the local variable in the function's scope. The original object outside the function remains unchanged.

In [151]:
def modify(x: int):
    x = 10
    print("x inside function:", x)

x = 5
modify(x)
print("x outside function:", x)


x inside function: 10
x outside function: 5


### Mutable Objects

Mutable objects (e.g., lists, dictionaries, sets) can be changed after they are created. When you pass a mutable object to a function, and the function modifies the object (e.g., by adding an item to a list), the changes will be reflected in the original object outside the function, because both the outside and inside variables refer to the same object in memory.


In [152]:
def modify(lst):
    lst.append(4)
    print("lst inside function:", lst)

lst = [1, 2, 3]
modify(lst)
print("lst outside function:", lst)

lst inside function: [1, 2, 3, 4]
lst outside function: [1, 2, 3, 4]


This behavior shows that Python's model is not purely call by value (since changes to mutable objects within a function are seen outside the function) nor purely call by reference (since reassigning an object within a function does not reassign the external reference). Instead, Python passes the references to the objects by value, a subtle distinction that underscores the importance of understanding mutable and immutable objects in Python.

## MRO

MRO stands for Method Resolution Order in Python. It is the order in which Python looks for a method in a hierarchy of classes. Especially with multiple inheritance, where a class can inherit from multiple classes, Python needs to have a specific order in which it searches for methods to ensure that the correct method is called. Python uses the C3 linearization algorithm to determine this order, ensuring a consistent and predictable method resolution path.

The MRO defines the class search path used by Python to search for the right method to use in classes involved in multiple inheritance. The algorithm that Python uses to determine this order is aimed at preserving the following two properties:

1. **Child precedes its parents**: In the MRO of a class, the class itself is always listed before its parents.
2. **Base class order is preserved**: In the MRO of a class, the order of base classes as specified in the class definition is preserved.

You can determine the MRO of a class by using the `.__mro__` attribute or the `mro()` method on the class. Here's an example:

In [153]:
class A:
    pass

class B(A):
    pass

class C(A):
    pass

class D(B, C):
    pass

print(D.__mro__)
# Or equivalently
print(D.mro())

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]


## References in Python

In Python, references are used to access objects stored in memory. When you create a variable, you are creating a reference to an object. There are two primary types of references: strong references and weak references.

### Strong References

By default, all references in Python are strong references. This means that as long as there is at least one strong reference to an object, the object will not be garbage collected.

**Step-by-Step Breakdown**

**Object Creation:** When an object is created, it is allocated in memory.

**Strong Reference:** Any variable that references this object creates a strong reference.

**Reference Counting:** Python keeps track of the number of strong references to each object using a reference count.

**Garbage Collection:** When the reference count drops to zero (i.e., no strong references), the object is eligible for garbage collection.

In [166]:
import sys

class MyClass:
    def __init__(self, value):
        self.value = value

# Creating a strong reference
obj = MyClass(10)

# Checking the number of strong references to obj
# The output is 2 because:
# 1. The reference from obj.
# 2. The temporary reference from sys.getrefcount.
print(f'Reference count after creation: {sys.getrefcount(obj)}')  

# Creating another strong reference
another_obj = obj

# Checking the reference count again
# The output is 3 because:
# 1. The reference from obj.
# 2. The reference from another_obj.
# 3. The temporary reference from sys.getrefcount.
print(f'Reference count after adding another reference: {sys.getrefcount(obj)}') 

# Accessing the value through both references
print(f'Value accessed via obj: {obj.value}') 
print(f'Value accessed via another_obj: {another_obj.value}')  

# Deleting one strong reference
del obj

# Checking the reference count after deletion
# The output is 2 because:
# 1. The reference from another_obj.
# 2. The temporary reference from sys.getrefcount.
print(f'Reference count after deleting obj: {sys.getrefcount(another_obj)}')  

# Deleting the last strong reference
del another_obj

# Now the object is eligible for garbage collection
# No way to check reference count as object is no longer accessible


Reference count after creation: 2
Reference count after adding another reference: 3
Value accessed via obj: 10
Value accessed via another_obj: 10
Reference count after deleting obj: 2


### Weak References

Weak references allow the referenced object to be garbage collected even if the weak reference exists. They are useful for caching and other scenarios where you want to hold references to objects without preventing their garbage collection.

To create weak references, you need the `weakref` module.

In [167]:
import weakref

class MyClass:
    def __init__(self, value):
        self.value = value

# Creating a strong reference
obj = MyClass(10)

# Creating a weak reference to the variable
weak_obj = weakref.ref(obj)

print(f'Value accessed via weak reference: {weak_obj().value}') 

# Deleting the strong reference
del obj

# Checking if the weak reference is still valid
print(f"Weak reference value is: {weak_obj()}")

Value accessed via weak reference: 10
Weak reference value is: None


In this example, `weak_obj` is a weak reference to the `MyClass` instance. After deleting `obj`, the object can be garbage collected, and `weak_obj()` returns `None`.

Weak references are particularly useful in situations where you want to avoid memory leaks by not holding on to objects longer than necessary. Here are some common use cases:

#### Caching

Weak references can be used to implement caches that do not prevent objects from being garbage collected.

In [168]:
import weakref

class Data:
    def __init__(self, value):
        self.value = value

# Cache dictionary with weak references
cache = weakref.WeakValueDictionary()

# Creating instances of Data
data1 = Data(1)
data2 = Data(2)

# Adding objects to the cache with weak references
cache['key1'] = data1
cache['key2'] = data2

# Accessing and printing the value from the cache using key1
# This works because data1 is still a strong reference
print(f"Value for 'key1' in cache before deletion: {cache['key1'].value}")

# Deleting strong references to the Data instances
del data1
del data2

# Attempting to access the cached objects
# The objects should be garbage collected since there are no strong references left
print(f"Value for 'key1' in cache after deletion: {cache.get('key1')}")  # Expecting None if garbage collected
print(f"Value for 'key2' in cache after deletion: {cache.get('key2')}")  # Expecting None if garbage collected

Value for 'key1' in cache before deletion: 1
Value for 'key1' in cache after deletion: None
Value for 'key2' in cache after deletion: None


#### Tracking Object Lifecycle

Weak references can be used to monitor when objects are about to be destroyed, which can be useful for debugging or managing resources.

In [169]:
import weakref

class MyClass:
    def __init__(self, value):
        self.value = value

def on_finalize(weak_ref):
    # Check if the weak reference is still valid before accessing its value
    obj = weak_ref()
    if obj is not None:
        print(f'Object with value {obj.value} is about to be destroyed.')
    else:
        print('The object has already been destroyed.')

obj = MyClass(10)

# Creating a weak reference to obj with a callback function on_finalize
# The callback will be called when the object is about to be destroyed
weak_obj = weakref.ref(obj, on_finalize)

# Deleting the strong reference obj
# Since there are no other strong references to the MyClass instance,
# it will be garbage collected, and the on_finalize callback will be triggered
del obj

The object has already been destroyed.


#### Weak References to Methods

Weak references can also be used to reference instance methods, which can be particularly useful for event handling or callback systems.

In [170]:
import weakref

class MyClass:
    def method(self):
        print('Method called')

obj = MyClass()

# Create a weak reference to obj's method using weakref.WeakMethod
weak_method = weakref.WeakMethod(obj.method)

# Calling the weakly referenced method if it's still valid
if weak_method():
    # The weak reference is still valid, so call the method
    weak_method()()
else:
    print("The method is no longer available")

# Deleting the strong reference to the MyClass instance
del obj

# Check if the weakly referenced method is still callable
if weak_method():
    # The weak reference is still valid, so call the method
    weak_method()()
else:
    print("The method is no longer available")

Method called
The method is no longer available


#### Weak Sets

The `weakref` module also provides `WeakSet`, which is a set that holds weak references to its elements. This can be useful for keeping track of objects without preventing their garbage collection.

In [165]:
import weakref

class MyClass:
    def __init__(self, value):
        self.value = value

# Creating a WeakSet
weak_set = weakref.WeakSet()

obj1 = MyClass(1)
obj2 = MyClass(2)

# Adding objects to the WeakSet
weak_set.add(obj1)
weak_set.add(obj2)

# Printing the length of the WeakSet
# At this point, the set contains two objects
print(f"Length of weak_set before deletion: {len(weak_set)}")

# Deleting strong references to the objects
del obj1
del obj2

# Printing the length of the WeakSet after deletion
# If the objects have been garbage collected, the set should be empty
print(f"Length of weak_set after deletion: {len(weak_set)}")


Length of weak_set before deletion: 2
Length of weak_set after deletion: 0


## Enumerations

Enumerations, or Enums, in Python are a powerful feature that can make your code more readable and maintainable. Enums allow you to define a set of named values, making your code more expressive and avoiding the use of magic numbers or strings.

### 1. Introduction

Enums are a way to define symbolic names for a set of values. They are used to create a distinct type with a set of named values, which are known as members.

You can create an Enum by subclassing the `Enum` class and adding class attributes to it. Each attribute becomes an Enum member.

In [183]:
from enum import Enum

class Status(Enum):
    NEW = 'new'
    IN_PROGRESS = 'in_progress'
    COMPLETED = 'completed'

### 2. Accessing Enum Members

You can access Enum members using the dot notation or by value.

In [184]:
# Access by name
print(Status.NEW)

# Access by value
print(Status('in_progress')) 

# Access name and value
print(Status.NEW.name)  
print(Status.NEW.value)

Status.NEW
Status.IN_PROGRESS
NEW
new


### 3. Iteration

You can iterate over the members of an Enum.

In [185]:
for status in Status:
    print(status)

Status.NEW
Status.IN_PROGRESS
Status.COMPLETED


### 4. Comparison

Enum members are unique and can be compared using identity and equality operators.

In [186]:
print(Status.NEW == Status.NEW) 
print(Status.NEW is Status.NEW) 
print(Status.NEW == Status.COMPLETED)

True
True
False


### 5. Auto Value Assignment

You can use the `auto()` function for automatic value assignment.

In [187]:
from enum import Enum, auto

class Priority(Enum):
    LOW = auto()
    MEDIUM = auto()
    HIGH = auto()
    
print(Priority.LOW.value)
print(Priority.MEDIUM.value)
print(Priority.HIGH.value)

1
2
3


### 6. Using Enums in Classes

Enums can be used within classes to provide meaningful constants. It can also have methods.

In [188]:
from enum import Enum

class Status(Enum):
    NEW = 'new'
    IN_PROGRESS = 'in_progress'
    COMPLETED = 'completed'

    def describe(self):
        return f"Status is {self.name}, value is {self.value}"

class Task:
    def __init__(self, title, status):
        self.title = title
        self.status = Status(status)

    def change_status(self, new_status):
        if isinstance(new_status, Status):
            self.status = new_status
        else:
            raise ValueError("Invalid status")

    def status_description(self):
        return self.status.describe()

# Creating a new task with 'NEW' status
task = Task("Write documentation", Status.NEW)
print(f"Task: {task.title}, Status: {task.status.name}")
print(task.status_description())  

# Changing the task status to 'IN_PROGRESS'
task.change_status(Status.IN_PROGRESS)
print(f"Task: {task.title}, Status: {task.status.name}")
print(task.status_description())  # 

# Completing the task by changing its status to 'COMPLETED'
task.change_status(Status.COMPLETED)
print(f"Task: {task.title}, Status: {task.status.name}")
print(task.status_description())  

# Attempting to change to an invalid status (This will raise an error)
try:
    task.change_status("archived")
except ValueError as e:
    print(e) 


Task: Write documentation, Status: NEW
Status is NEW, value is new
Task: Write documentation, Status: IN_PROGRESS
Status is IN_PROGRESS, value is in_progress
Task: Write documentation, Status: COMPLETED
Status is COMPLETED, value is completed
Invalid status


### 7. Dynamic Enums

You can create Enums dynamically using the `Enum` class.

In [189]:
DynamicEnum = Enum('DynamicEnum', 'ONE TWO THREE')

### Examples

#### Command Pattern

Using Enums to define commands in the Command Pattern.

In [190]:
from enum import Enum

class Command(Enum):
    """Enum representing possible commands for the RemoteControl."""
    START = 'start'
    STOP = 'stop'
    PAUSE = 'pause'
    REWIND = 'rewind'
    FORWARD = 'forward'

class RemoteControl:
    """Class representing a remote control that can execute commands."""

    def __init__(self):
        # Mapping commands to their respective methods for execution
        self.command_methods = {
            Command.START: self.start,
            Command.STOP: self.stop,
            Command.PAUSE: self.pause,
            Command.REWIND: self.rewind,
            Command.FORWARD: self.forward
        }

    def execute(self, command):
        """
        Execute the given command if it exists in the command_methods dictionary.
        
        :param command: Command enum member
        """
        if not isinstance(command, Command):
            raise ValueError("Invalid command type")

        # Get the corresponding method for the command
        action = self.command_methods.get(command)
        
        if action:
            action()  # Call the method
        else:
            print("Unknown command")

    def start(self):
        """Action for starting."""
        print("Starting...")

    def stop(self):
        """Action for stopping."""
        print("Stopping...")

    def pause(self):
        """Action for pausing."""
        print("Pausing...")

    def rewind(self):
        """Action for rewinding."""
        print("Rewinding...")

    def forward(self):
        """Action for forwarding."""
        print("Forwarding...")

remote = RemoteControl()

# Execute various commands using the remote control
remote.execute(Command.START)    # Execute the START command
remote.execute(Command.PAUSE)    # Execute the PAUSE command
remote.execute(Command.REWIND)   # Execute the REWIND command
remote.execute(Command.FORWARD)  # Execute the FORWARD command
remote.execute(Command.STOP)     # Execute the STOP command

# Attempt to execute an invalid command (this will raise an error)
try:
    remote.execute("invalid_command")
except ValueError as e:
    print(e)  # Handling the invalid command case


Starting...
Pausing...
Rewinding...
Forwarding...
Stopping...
Invalid command type


#### Configuration Management

Using Enums for configuration settings.

In [191]:
from enum import Enum

class Config(Enum):
    """Enum representing different configuration settings for the application."""
    DEBUG = 'debug'
    PRODUCTION = 'production'
    TESTING = 'testing'

class Application:
    """Class representing an application that operates under different configurations."""

    def __init__(self, config):
        """
        Initialize the application with the given configuration.
        
        :param config: Config enum member
        """
        if not isinstance(config, Config):
            raise ValueError("Invalid configuration type")
        self.config = config
        self.setup()

    def setup(self):
        """Set up the application based on the configuration."""
        if self.config == Config.DEBUG:
            self.enable_debug_mode()
        elif self.config == Config.PRODUCTION:
            self.enable_production_mode()
        elif self.config == Config.TESTING:
            self.enable_testing_mode()

    def enable_debug_mode(self):
        """Set up the application in debug mode."""
        self.debug = True
        self.logging_level = 'DEBUG'
        print("Debug mode enabled.")

    def enable_production_mode(self):
        """Set up the application in production mode."""
        self.debug = False
        self.logging_level = 'ERROR'
        print("Production mode enabled.")

    def enable_testing_mode(self):
        """Set up the application in testing mode."""
        self.debug = True
        self.logging_level = 'INFO'
        print("Testing mode enabled.")

    def get_config_description(self):
        """Get a description of the current configuration."""
        return f"Current configuration: {self.config.name}, Logging level: {self.logging_level}"

# Create an application instance with production configuration
app = Application(Config.PRODUCTION)
print(app.get_config_description())  # Output the current configuration description

# Create an application instance with debug configuration
app_debug = Application(Config.DEBUG)
print(app_debug.get_config_description())  # Output the current configuration description

# Create an application instance with testing configuration
app_testing = Application(Config.TESTING)
print(app_testing.get_config_description())  # Output the current configuration description

# Attempt to create an application with an invalid configuration (this will raise an error)
try:
    app_invalid = Application("invalid_config")
except ValueError as e:
    print(e)  # Handling the invalid configuration case

Production mode enabled.
Current configuration: PRODUCTION, Logging level: ERROR
Debug mode enabled.
Current configuration: DEBUG, Logging level: DEBUG
Testing mode enabled.
Current configuration: TESTING, Logging level: INFO
Invalid configuration type
